In [29]:
import pandas as pd
df=pd.read_excel('Dcard.xlsx', header=0)  

In [67]:
df=df.dropna()
df.head(5)

,Unnamed: 0,title,content,createdAt,depression
1,1.0,我的故事還在堅持 #第一章,嗨，妳好，展開笑容揮手。這一向是我面對朋友親戚外人的第一反應。我想打完我至今的故事。因為也許...,2018-09-30T16:44:54.678Z,1
2,2.0,反正你們不會知道,最近陷入低潮，這低潮沒有人知道因為我還沒有勇氣對身邊的人提起人群恐慌症重度憂鬱症光是練習說出...,2019-05-10T14:01:24.268Z,1
3,3.0,快樂是什麼？,本人20歲，是個重度憂鬱症的患者曾經自殺被阻止過，也好幾個月都不出門由於家庭因素、國小(中)...,2018-10-18T08:02:50.729Z,1
4,4.0,我是重鬱者，我復發了！,去年的時候，我自殺了。最近，家裡鬧得很兇，很厲害。我選擇封鎖，因為每天，我的手機一直響。這舉...,2018-11-29T08:29:39.523Z,1
5,5.0,如何幫助重度憂鬱症？,男友是一個重度憂鬱.焦慮症的患者...有去看過身心科了但藥量一直增加（大概持續吃一年多了）常...,2019-10-29T12:34:41.719Z,1


In [68]:
df['content']=df['content'].apply(lambda x: x.replace('\n',''))

In [69]:
from ckiptagger import data_utils
from ckiptagger import WS, POS, NER
ws=WS("C:/Users/Lenovo/Documents/code/fb_Carrefour/data")
pos=POS("C:/Users/Lenovo/Documents/code/fb_Carrefour/data")
ner=NER("C:/Users/Lenovo/Documents/code/fb_Carrefour/data")

In [70]:
def about_ner(text):
    
    if str(text)=='nan':
        return ''
    
    ws_results=ws([text])
    pos_results=pos(ws_results)
    
    res=[]
    for i in zip(ws_results[0], pos_results[0]):
        if (i[1]=='Na' or i[1]=='Nb' or i[1]=='Nc' or i[1]=='ORG' or i[1]=='GPE' or i[1]=='FAC' or\
            i[1]=='NORP' or i[1]=='PERSON' or i[1]=='VC'):
            res.append(i[0])
    return res

In [71]:
res=[]
res.append(df['content'].apply(lambda x: ','.join(about_ner(x))))

In [128]:
res[0]

1     展開,笑容,面對,朋友,親戚,外人,反應,打完,故事,選擇,人,事,閨蜜,訊息,風格,事情,...
2     低潮,低潮,人,勇氣,身邊,人,人群,恐慌症,憂鬱症,練習,字,眼淚,控制,學生,時期,言語...
3     憂鬱症,患者,阻止,月,家庭,因素,國小,單親,家庭,主義,者,凡事,心,世上,壓力,放,身...
4     時候,家,鬧,選擇,封鎖,手機,舉動,引來,親戚,反應,批判,做,事情,選擇,親戚,封鎖,家...
5     男友,焦慮症,患者,看過,身心科,藥量,吃,胸,心臟痛,吃,止痛藥,男友,可能,氣,醫生,幫...
6     揮,手,展開,笑顏,前提,補充,媽,精神,狀態,人,舉動,哥哥,人,人,人,”,插,手管,事...
7                                          重鬱症,權利,藥物,輔助
8        世界,人,意外,人,時候,病痛,折磨,呼吸,浪費,生命,人,人,拜託,夢,人,祝福,病痛,身
9                 老師,條件,學校,呼吸,位子,宿舍,換,室友,人,世界,末日,找,工作,做
10    人們,到,光芒,找到,方法,做,傷到,凶宅,責怪,月,方法,人生,眼淚,面試,工作,時候,正...
11    寵物,靈性,病症,人,腦袋,胸,呼吸,心理,狀態,眼淚,眼眶,關,燈,床,貓貓,腳,踩,腳,...
12    世界,時候,精神科,醫生,精神病院,確診,憂鬱症,確診,躁鬱症,疾病,老師,誘姦,強暴,精神...
13    人,機會,歉,看,標題,點進來,希望,解,事,文章,解,撐,憂鬱症,心理,疾病,醫生,協助,...
Name: content, dtype: object

In [136]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(res[0])
#sg=0以CBOW來訓練
#sg=1以Skip-gram來訓練
#Skip-gram比CBOW通常對低頻詞有更好的訓練效果
model_d250 = Word2Vec(res[0], size=250, iter=10, sg=0)

#其他參數
#size=100 => 詞向量的維度大小，維度太小會無法有效表達詞與詞的關係，維度太大會使關係太稀疏而難以找出規則
#iter=5 => 訓練的回數，訓練過少會使得詞關係過為鬆散，訓練過度又會使得詞關係過為極端
#window=5 => CBOW下決定Word2Vec一次取五詞來預測中間詞，Skip-gram下決定Word2Vec用一個詞，找出附近的五個詞
#min_count=5 => 出現次數大於等於min_count的詞，才會納入Word2Vec的詞典中
#max_vocab_size=None => Word2Vec的詞典容納上限，出現次數最低的詞會優先被剔除
#negative=5 => Sampling的取樣數量，5~20適合小數據，2~5適合大數據
#workers=3 => 執行核心數量

In [130]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [133]:
most_similar(model, ['生病'])


生病 not found in Word2Vec model!


""


In [137]:
most_similar(model_d250, ['人','情','心'])

,人,cos,情,cos,心,cos
0,",",0.999920,",",0.999928,",",0.999921
1,情,0.999863,子,0.999874,情,0.999870
2,心,0.999863,心,0.999870,人,0.999863
3,眼,0.999862,人,0.999863,貓,0.999857
4,病,0.999852,病,0.999861,症,0.999853
5,子,0.999852,時,0.999856,舉,0.999853
6,生,0.999850,眼,0.999854,子,0.999852
7,症,0.999848,症,0.999846,時,0.999851
8,貓,0.999846,家,0.999845,生,0.999851
9,狀,0.999845,貓,0.999840,眼,0.999851


In [121]:
model_d250_sg = Word2Vec(word, size=250, iter=10, sg=1)
most_similar(model_d250_sg, ['人'])

,人,cos
0,",",0.999761
1,傷,0.999739
2,換,0.999731
3,狀,0.999726
4,學,0.999724
5,面,0.999721
6,幫,0.999719
7,身,0.999719
8,候,0.999717
9,眼,0.999712


In [134]:
#儲存模型
model.save('word2vec.model')

In [135]:
#讀取模型
model_load = Word2Vec.load('word2vec.model')